In [1]:
#https://github.com/seungeunrho/minimalRL/blob/master/ddpg.py

In [2]:
from unityagents import UnityEnvironment
import numpy as np

import random
import collections
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

torch.set_default_tensor_type('torch.DoubleTensor')

#Hyperparameters
lr_mu        = 0.0001
lr_q         = 0.0001
gamma        = 0.99
batch_size   = 128
buffer_limit = 100000
tau          = 0.001 # for target network soft update

class ReplayBuffer():
    def __init__(self):
        self.buffer = collections.deque(maxlen=buffer_limit)

    def put(self, transition):
        self.buffer.append(transition)
    
    def sample(self, n):
        mini_batch = random.sample(self.buffer, n)
        s_lst, a_lst, r_lst, s_prime_lst, done_mask_lst = [], [], [], [], []

        for transition in mini_batch:
            s, a, r, s_prime, done_mask = transition
            s_lst.append(s)
            a_lst.append([a])
            r_lst.append([r])
            s_prime_lst.append(s_prime)
            done_mask_lst.append([done_mask])
        
        return torch.tensor(s_lst, dtype=torch.float), torch.tensor(a_lst), \
               torch.tensor(r_lst), torch.tensor(s_prime_lst, dtype=torch.float), \
               torch.tensor(done_mask_lst)
    
    def size(self):
        return len(self.buffer)

class MuNet(nn.Module):
    def __init__(self):
        super(MuNet, self).__init__()
        self.fc1 = nn.Linear(33, 256)
        self.fc2 = nn.Linear(256, 128)
        self.fc_mu = nn.Linear(128, 4)

    def forward(self, x):
        x = F.relu(self.fc1(x.double()))
        x = F.relu(self.fc2(x.double()))
        mu = torch.tanh(self.fc_mu(x))
        return mu

class QNet(nn.Module):
    def __init__(self):
        super(QNet, self).__init__()
        
        self.fc_s = nn.Linear(33, 256)
        #self.fc_a = nn.Linear(4,  28)
        self.fc_q = nn.Linear(260, 128)
        self.fc_3 = nn.Linear(128,  1)

    def forward(self, x, a):
        h1 = self.fc_s(x.double())
        h1 = F.relu(h1)
        
        a = torch.squeeze(a.double())
        #h2 = self.fc_a(a)
        #h2 = F.relu(h2)

        cat = torch.cat([h1, a], dim=1)
        q = F.relu(self.fc_q(cat))
        q = self.fc_3(q)
        return q

class OrnsteinUhlenbeckNoise:
    def __init__(self, mu):
        self.theta, self.dt, self.sigma = 0., 0.0, 0.
        self.mu = mu
        self.x_prev = np.zeros_like(self.mu)

    def __call__(self):
        x = self.x_prev + self.theta * (self.mu - self.x_prev) * self.dt + \
                self.sigma * np.sqrt(self.dt) * np.random.normal(size=self.mu.shape)
        self.x_prev = x
        return x
      
def train(mu, mu_target, q, q_target, memory, q_optimizer, mu_optimizer):
    s,a,r,s_prime,done_mask  = memory.sample(batch_size)
    
    target = r + gamma * q_target(s_prime, mu_target(s_prime))
    q_loss = F.smooth_l1_loss(q(s,a), target.detach())
    q_optimizer.zero_grad()
    q_loss.backward()
    q_optimizer.step()
    
    mu_loss = -q(s,mu(s)).mean() # That's all for the policy loss.
    mu_optimizer.zero_grad()
    mu_loss.backward()
    mu_optimizer.step()
    
def soft_update(net, net_target):
    for param_target, param in zip(net_target.parameters(), net.parameters()):
        param_target.data.copy_(param_target.data * (1.0 - tau) + param.data * tau)
    

In [3]:
env = UnityEnvironment(file_name='Reacher.app')

# get the default brain
brain_name = env.brain_names[0]
brain = env.brains[brain_name]

memory = ReplayBuffer()

q, q_target = QNet(), QNet()
q_target.load_state_dict(q.state_dict())
mu, mu_target = MuNet(), MuNet()
mu_target.load_state_dict(mu.state_dict())

mu.load_state_dict(torch.load('mu.pth'))
mu_target.load_state_dict(torch.load('mu.pth'))
q.load_state_dict(torch.load('q.pth'))
q_target.load_state_dict(torch.load('q.pth'))

score = 0.0
print_interval = 1

mu_optimizer = optim.Adam(mu.parameters(), lr=lr_mu)
q_optimizer  = optim.Adam(q.parameters(), lr=lr_q)
ou_noise = OrnsteinUhlenbeckNoise(mu=np.zeros(4))

for n_epi in range(1000):
    env_info = env.reset(train_mode=False)[brain_name]
    s = env_info.vector_observations

    while True: # maximum length of episode is 200 for Pendulum-v0
        a = mu(torch.from_numpy(s)) 
        a = a.detach().numpy() + ou_noise()
        
        env_info = env.step(a)[brain_name]
        s_prime = env_info.vector_observations
        r = env_info.rewards[0]
        done = env_info.local_done
        
        #memory.put((s[0], a[0], r,s_prime[0], done))
        score += r
        s = s_prime
        
        if False:
            train(mu, mu_target, q, q_target, memory, q_optimizer, mu_optimizer)
            
            soft_update(mu, mu_target)
            soft_update(q,  q_target)
            
        if np.any(done):
            #print(f'episode {n_epi}: {score}')
            break
    
    if n_epi%print_interval==0 and n_epi!=0:
        print("# of episode :{}, avg score : {:.1f}".format(n_epi, score/print_interval))
        score = 0.0

env.close()

INFO:unityagents:
'Academy' started successfully!
Unity Academy name: Academy
        Number of Brains: 1
        Number of External Brains : 1
        Lesson number : 0
        Reset Parameters :
		goal_speed -> 1.0
		goal_size -> 5.0
Unity brain name: ReacherBrain
        Number of Visual Observations (per agent): 0
        Vector Observation space type: continuous
        Vector Observation space size (per agent): 33
        Number of stacked Vector Observation: 1
        Vector Action space type: continuous
        Vector Action space size (per agent): 4
        Vector Action descriptions: , , , 
ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.

INFO:root:
Unfortunately, your original traceback can not be constructed.



Traceback (most recent call last):
  File "/Users/pablot/anaconda3/envs/drlnd/lib/python3.6/site-packages/IPython/core/interactiveshell.py", line 3331, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-3-125d10316b1b>", line 34, in <module>
    env_info = env.step(a)[brain_name]
  File "/Users/pablot/anaconda3/envs/drlnd/lib/python3.6/site-packages/unityagents/environment.py", line 369, in step
    self._generate_step_input(vector_action, memory, text_action)
  File "/Users/pablot/anaconda3/envs/drlnd/lib/python3.6/site-packages/unityagents/rpc_communicator.py", line 78, in exchange
    output = self.unity_to_external.parent_conn.recv()
  File "/Users/pablot/anaconda3/envs/drlnd/lib/python3.6/multiprocessing/connection.py", line 250, in recv
    buf = self._recv_bytes()
  File "/Users/pablot/anaconda3/envs/drlnd/lib/python3.6/multiprocessing/connection.py", line 407, in _recv_bytes
    buf = self._recv(4)
  File "/Users/pablot/anaconda3/envs/drlnd

KeyboardInterrupt: 

In [ ]:
q